<a href="https://colab.research.google.com/github/changsin/AI/blob/main/rl_value_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Value Iteration

In reinforcement learning, to find the optimal policy two methods are used: value iteration or policy iteration. Using the value iteration method, the optimal policy is calculated using Bellman's equation:

$$ \pi^*(s) = \underset{a} argmax[r(s, a) + \gamma V^*(\delta(s, a))] $$

- $ $$ \pi^*(s) $: optimal policy
- $ r(s, a) $: immediate reward
- $ \gamma $: discount factor
- $ V^*(\delta(s, a)) $: future reward

The optimal policy results in the maximum value $ V^* $:
$$ \hat V(s) = \underset{a} max[r(s, a) + \gamma V^*(\delta(s, a))] $$

Using the equation, the state value is updated till there is no more update needed. The following code implements the algorithm using the example in p. 297 of Ertel (2017).

In [ ]:
import numpy as np
from enum import Enum

class Action:
  def __init__(from_pos, move, reward):
    this.from_pos = from_pos
    this.move = move
    this.reward = reward

class Move(Enum):
  L = 1
  R = 2
  U = 3
  D = 4

class Grid:
  # to force convergence, we consider the values are converged
  # if the updated value is less than the precision value
  PRECISION = 0.001
  MOVES = [Move.L, Move.R, Move.U, Move.D]

  def __init__(self, state, actions, gamma = 0.9, rows=3, columns=3):
    self.state = state
    self.actions = actions
    self.gamma = gamma
    self.rows = rows
    self.columns = columns
  
  def __repr__(self):
    rows = ""
    for row in range(self.rows):
      columns = ""
      for col in range(self.columns):
        columns += " {:03.02f} ".format(self.state[row][col])
      rows += "\n" + columns
      # print(row_values)
    return rows

  def converge(self, limit=100):
    for i in range(limit):
      if self.update_state() == 0:
        print("Converged at ", i)
        return
    
    print("Did not converge after ", limit)

  def update_state(self):
    updated = 0
    # update the state from bottom to top, left to right
    for row in range(self.rows - 1, -1, -1):
      for col in range(0, self.columns, 1):
        value_original = self.state[row][col]
        self.state[row][col] = self.calc_optimal_value((row, col))

        if abs(value_original - self.state[row][col]) > self.PRECISION:
          updated += 1
    print(self)
    return updated
  
  def to_target_pos(self, pos, move):
    cur_row, cur_col = pos
    if Move.L == move:
      return (cur_row, cur_col - 1)
    elif Move.R == move:
      return (cur_row, cur_col + 1)
    elif Move.U == move:
      return (cur_row - 1, cur_col)
    elif Move.D == move:
      return (cur_row + 1, cur_col)
    else:
      raise Exception("Invalid move")

  def is_valid_move(self, pos, move):
    target_row, target_col = self.to_target_pos(pos, move)
    return target_row < self.rows and target_col < self.columns and \
      target_row >= 0 and target_col >= 0

  def get_immediate_reward(self, pos, move):
    return self.actions[pos, move] if (pos, move) in self.actions else 0

  def get_state_value(self, pos):
    return self.state[pos[0]][pos[1]]

  def calc_optimal_value(self, pos):
    """
    pi*(s) = argmax[r(s, a) + \gamma V*(\delta(s, a))]
    """
    possible_values = []
    for move in self.MOVES:
      if self.is_valid_move(pos, move):
        target_pos = self.to_target_pos(pos, move)
        value = self.get_immediate_reward(pos, move) + self.gamma*self.get_state_value(target_pos)
        possible_values.append(value)
    return max(possible_values)

In [ ]:
grid_state = [[0, 0, 0],
              [0, 0, 0],
              [0, 0, 0]]

actions = {}
actions[(2, 0), Move.U] = -1
actions[(2, 0), Move.R] = -1
actions[(2, 1), Move.R] = -1
actions[(2, 1), Move.L] = 1
actions[(2, 2), Move.L] = 1


grid = Grid(grid_state, actions)
# print(grid)
# grid.update_state()
grid.converge()

In [ ]:
grid_state1 = [[0, 0, 0],
              [0, 0, 0],
              [0, 0, 0]]

actions1 = {}
actions1[(1, 1), Move.R] = 40
actions1[(2, 2), Move.R] = 100
actions1[(0, 2), Move.L] = 70


grid1 = Grid(grid_state1, actions1)
# print(grid)
# grid.update_state()
grid1.converge()

# Q Learning in Gym

In [ ]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import random
from IPython.display import clear_output

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
#Fixing seed for reproducibility
np.random.seed(0)
#Loading and rendering the gym environment
env = gym.make("Taxi-v3").env
env.reset()
env.render()

In [ ]:
#Getting the state space
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

#STEP 1 - Initializing the Q-table

Q = np.zeros((env.observation_space.n, env.action_space.n))

#Setting the hyperparameters
              
alpha = 0.7 #learning rate                 
discount_factor = 0.618               
epsilon = 1                  
max_epsilon = 1
min_epsilon = 0.01         
decay = 0.01         

train_episodes = 2000    
test_episodes = 100          
max_steps = 100 

In [ ]:
#Training the agent

#Creating lists to keep track of reward and epsilon values
training_rewards = []  
epsilons = []

for episode in range(train_episodes):
    #Reseting the environment each time as per requirement
    state = env.reset()    
    #Starting the tracker for the rewards
    total_training_rewards = 0
    
    for step in range(100):
        #Choosing an action given the states based on a random number
        exp_exp_tradeoff = random.uniform(0, 1) 
        
        
        ### STEP 2: SECOND option for choosing the initial action - exploit     
        #If the random number is larger than epsilon: employing exploitation 
        #and selecting best action 
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(Q[state,:])      
            
        ### STEP 2: FIRST option for choosing the initial action - explore       
        #Otherwise, employing exploration: choosing a random action 
        else:
            action = env.action_space.sample()
            
            
        ### STEPs 3 & 4: performing the action and getting the reward     
        #Taking the action and getting the reward and outcome state
        new_state, reward, done, info = env.step(action)
        

        ### STEP 5: update the Q-table
        #Updating the Q-table using the Bellman equation
        Q[state, action] = Q[state, action] + alpha * (reward + discount_factor * np.max(Q[new_state, :]) - Q[state, action]) 
        #Increasing our total reward and updating the state
        total_training_rewards += reward      
        state = new_state         
        
        #Ending the episode
        if done == True:
            #print ("Total reward for episode {}: {}".format(episode, total_training_rewards))
            break
    
    #Cutting down on exploration by reducing the epsilon 
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay*episode)
    
    #Adding the total reward and reduced epsilon values
    training_rewards.append(total_training_rewards)
    epsilons.append(epsilon)

print ("Training score over time: " + str(sum(training_rewards)/train_episodes))

In [ ]:
env.render()

In [ ]:
#Visualizing results and total reward over all episodes
x = range(train_episodes)
plt.plot(x, training_rewards)
plt.xlabel('Episode')
plt.ylabel('Training total reward')
plt.title('Total rewards over all episodes in training') 
plt.show()

#Visualizing the epsilons over all episodes
plt.plot(epsilons)
plt.xlabel('Episode')
plt.ylabel('Epsilon')
plt.title("Epsilon for episode")
plt.show()

In [ ]:
import gym
import numpy as np

#create environment
env = gym.make("Taxi-v3")

#initialize Q-table
Q = np.zeros([env.observation_space.n, env.action_space.n])

gamma = 0.1
alpha = 0.1
epsilon = 0.1
epsilon_decay = 0.99 #decay factor 


total_epochs = 0
episodes = 10000

for episode in range(episodes):
    epochs = 0
    reward = 0
    epsilon = epsilon * epsilon_decay #decay step

    state = env.reset()
    
    while reward != 20:
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])
        next_state, reward, done, info = env.step(action)
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * \
                            np.max(Q[next_state]) - Q[state, action])
        state = next_state 
        epochs += 1
    total_epochs += epochs
    
print("Average timesteps taken: {}".format(total_epochs/episodes))